## PRF initial choice
the PRF is chosen to match the 3 dB elevation beam-width and aligned to center the swath between pulses
1. we start finding the approximated value for the beam extremes elevation angles:
    \begin{equation}
        \begin{split}
        \theta_1 = \theta - \dfrac{\lambda}{2 W_a}\\
        \theta_2 = \theta + \dfrac{\lambda}{2 W_a}
        \end{split}
    \end{equation}
2. we can use this approximate incidence angles to find the two slant range points.
    This can be done with the function "range_from_theta(theta)" that numerically
    inverts the slant range for an incidence angle on a point over a spherical earth.
    we'll then have:
    \begin{equation}
        \begin{split}
        R_1 = \text{range_from_theta}(\theta_1)\\
        R_2 = \text{range_from_theta}(\theta_2)
        \end{split}
    \end{equation}
3. The Pulse Repetition Interval ($1/\text{PRF}$) initial value is then found to match the slant range delta roundtrip delay
    \begin{equation}
        \text{PRI} = \dfrac{ 2 (R_2 - R_1) }{c}
    \end{equation}
4. To "center" the pulse in the received signal, the near end slant range round trip delay must be an integer multiple of the PRI.
    We therefore need to find the impulse order closest to the one associated to $R_1$ and recompute $R_1$, $R_2$, and PRI to the nearest *legal* value.
    \begin{equation}
        n = \text{floor}\left(\dfrac{2 R }{c \text{PRI}}\right)
    \end{equation}\
        Where R is the average slant range
5. a new PRI is chosen to place the beam center point at the center of the scene:
    \begin{equation}
        \text{PRI}' = \dfrac{2 R}{(n + 0.5) c}
    \end{equation}
6. From the new PRI we find the corrected $R_1$, $R_2$ and slant range swath width
       \begin{equation}
        \begin{split}
        R_1' = n \text{PRI} c /2\\
        R_2' = (n+1) \text{PRI} c /2\\
        \end{split}
    \end{equation}
7. From the corrected slant ranges we can find the ground NE and FE ranges and swath width

In [9]:
import numpy as np

# test parameters
theta = 30  # deg
wa = 0.3  # m

c = 299792458
freq = 10e9
# wavelength
wavel = c / freq

# # step 1 elevation limits
theta1 = (theta * np.pi / 180) + wavel / (2 * wa)
theta2 = (theta * np.pi / 180) - wavel / (2 * wa)

print(" elevation beamwidth = ", 180 / np.pi * wavel / wa, "deg")

print(" theta 1 = ", theta1 * 180 / np.pi, "deg")
print(" theta 2 = ", theta2 * 180 / np.pi, "deg")


 elevation beamwidth =  5.725614191084331 deg
 theta 1 =  32.86280709554216 deg
 theta 2 =  27.13719290445783 deg


In [10]:
from design_functions import range_from_theta

# # step 2 slant range limits
r1s, r1g = range_from_theta(theta1 * 180 / np.pi)
r2s, r2g = range_from_theta(theta2 * 180 / np.pi)

# slant swath
delta_r_s = r1s - r2s
delta_r_g = r1g - r2g
# ground swath
print("r1 = ", r1s[0], ' m')
print("r2 = ", r2s[0], ' m')
print("ground swath = ", delta_r_g, ' m')
print("slant range swath = ", delta_r_s, ' m')


r1 =  586479.046921434  m
r2 =  556578.3381809548  m
ground swath =  [59740.35029745]  m
slant range swath =  [29900.70874048]  m


In [11]:
# # step 3 intial PRI
PRI = float(2 * delta_r_s / c)
print("initial PRI = ", PRI, " s")

initial PRI =  0.00019947605713602838  s


In [12]:
# # step 4 impulse order determination
# average slant range
rs = np.average((r1s, r2s))
order = np.floor(2 * rs / (PRI * c))
print("average slant range = ", rs, " m")
print("pulse order = ", order)

average slant range =  571528.6925511945  m
pulse order =  19.0


In [13]:
# # step 5 adjusted PRI
PRI1 = 2 * rs / (c * (.5 + order))
print("adjusted PRI = ", PRI1, " s")

adjusted PRI =  0.0001955296935493559  s


In [14]:
# # step 6 adjusted near end and far end range and swath
r2s1 = order * c * PRI1 / 2
r1s1 = (order + 1) * c * PRI1 / 2
# slant range delta
delta_r_s_1 = r1s1 - r2s1
print("corrected slant range swath = ", delta_r_s_1, " m")

corrected slant range swath =  29309.163720574114  m


In [15]:
from design_functions import range_slant_to_ground

# # step 7 adjusted ground ranges
r1g1, theta1 = range_slant_to_ground(r1s1)
r2g1, theta2 = range_slant_to_ground(r2s1)
rg1 = r1g1 - r2g1
print("corrected ground swath = ", rg1, " m")
print("corrected theta1 = ", 180 / np.pi * theta1, " deg")
print("corrected theta2 = ", 180 / np.pi * theta2, " deg")

corrected ground swath =  58547.18012095246  m
corrected theta1 =  32.81311892660367  deg
corrected theta2 =  27.20232409144744  deg


## in a concise function:

In [16]:
from design_functions import pri_max_swath

PRI_1, ground_swath = pri_max_swath(theta, wa, freq=freq)
print("corrected PRI = ", PRI_1)
print("corrected PRF = ", 1 / PRI_1)
print("corrected ground swath = ", ground_swath)

corrected PRI =  0.0001955296935493559
corrected PRF =  5114.312725844776
corrected ground swath =  58547.18012095246
